In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project="prod-organize-arizon-4e1c0a83")

query = """
SELECT
  c.county,
  c.pctnum,
  b.congressionaldistrict,
  b.statehousedistrict,
  a.catalistmodel_income_bin,
  COUNT(a.dwid) AS DWIDS
FROM `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__models` as a
LEFT JOIN `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__district` as b
  ON a.dwid = b.dwid
INNER JOIN `prod-organize-arizon-4e1c0a83.rich_christina_proj.catalist_pctnum_crosswalk_native` as c
  ON b.uniqueprecinctcode = c.uniqueprecinctcode
WHERE b.state = "AZ"
GROUP BY 1, 2, 3, 4, 5
"""

# Runs query and returns pandas DataFrame
results_df = client.query(query).to_dataframe()


In [ ]:
results_df.head()

In [ ]:
pivot_income_df = results_df.pivot_table(
    index=['pctnum', 'county', 'congressionaldistrict', 'statehousedistrict'],
    columns='catalistmodel_income_bin',
    values='DWIDS',
    fill_value=0
).astype(int).reset_index()

pivot_income_df.head()

In [ ]:
# Reorder columns
income_order = [
    'Less than $20,000',
    '$20,000 - $30,000',
    '$30,000 - $50,000',
    '$50,000 - $75,000',
    '$75,000 - $100,000',
    '$100,000 - $150,000',
    'Greater than $150,000'
]
index_cols = ['pctnum', 'county', 'congressionaldistrict', 'statehousedistrict']
pivot_df = pivot_income_df[index_cols + income_order]

# Sanitize column names for BigQuery
pivot_df.columns = (
    pivot_df.columns
    .str.replace(r'[^\w]+', '_', regex=True)
    .str.strip('_')
    .str.lower()
)



pivot_df.head()

In [ ]:
from pandas_gbq import to_gbq

# Replace with your actual project and temp dataset
project_id = "prod-organize-arizon-4e1c0a83"
dataset_table = "rich_christina_proj.income_counts_pivot"  # dataset.table

# Upload DataFrame to BigQuery
to_gbq(
    pivot_df,
    destination_table=dataset_table,
    project_id=project_id,
    if_exists='replace'  # or 'append'
)
